In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
from classes import *

mnist_data = datasets.MNIST('data', train = True, download = True,
                           transform = transforms.ToTensor())
mnist_data = list(mnist_data)[:4096]

***

Autoencoders with convolution and deconvolution layers

In [21]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            ConvBlock(1, 16, kernel_size = 3, stride = 2, padding = 1),
            ConvBlock(16, 32, kernel_size = 3, stride = 2, padding = 1),
            ConvBlock(32, 64, kernel_size = 7, activFunc = 'none')
        )
        self.decoder = nn.Sequential(
            DeconvBlock(64, 32, kernel_size = 7),
            DeconvBlock(32, 16, kernel_size = 3, stride = 2, padding = 1, output_padding = 1),
            DeconvBlock(16, 1,  kernel_size = 3, stride = 2, padding = 1, output_padding = 1, activFunc = 'sigmoid')
        )
    
    def forward(self, X):
        X = self.encoder(X)
        X = self.decoder(X)
        return X

In [53]:
def train(model, num_epochs = 5, batch_size = 64, learning_rate = 1e-3):
    torch.manual_seed(42)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr = learning_rate,
                                 weight_decay = 1e-5)
    train_loader = torch.utils.data.DataLoader(mnist_data,
                                               batch_size = batch_size,
                                               shuffle = True)
    outputs = []
    for epoch in range(num_epochs):
        for data in train_loader:
            img, _ = data
            recon = model(img)
            loss = criterion(recon, img)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
        outputs.append((epoch, img, recon),)
    
    return outputs

In [ ]:
model = Autoencoder().
max_epochs = 20
outputs = train(model, num_epochs = max_epochs)

In [ ]:
for k in range(0, max_epochs, 5):
    plt.figure(figsize = (9, 2))
    imgs = outputs[k][1].detach().numpy()
    recon = outputs[k][2].detach().numpy()
    for i, item in enumerate(imgs):
        if i >= 9: break
        plt.subplot(2, 9, i+1)
        plt.imshow(item[0])
            
    for i, item in enumerate(recon):
        if i >= 9: break
        plt.subplot(2, 9, 9+i+1)
        plt.imshow(item[0])

***

Autoencoder with upsampling

A compressed representation can be great for saving and sharing any kind of data in a way that is more efficient than storing raw data. In practice, the compressed representation often holds key information about an input image and we can use it for denoising images or oher kinds of reconstruction and transformation!

In [ ]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

transform = transforms.ToTensor()
train_data = datasets.MNIST(root = 'data', train = True,
                            download = True, transform = transform)
test_data  = datasets.MNIST(root = 'data', train = False,
                            download = True, transform = transform)

In [ ]:
numWorkers = 2
batchSize = 20

trainLoader = torch.utils.data.DataLoader(train_data, batch_size = batchSize,
                                          num_workers = numWorkers)
testLoader  = torch.utils.data.DataLoader(test_data,  batch_size = batchSize,
                                          num_workers = numWorkers)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

images, labels = iter(trainLoader).next()
images = images.numpy()

img = np.squeeze(images[0])

fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(111)
ax.imshow(img, cmap = 'gray')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        # encoder layers #
        self.conv1 = nn.Conv2d(1, 16, 3, padding = 1)
        self.conv2 = nn.Conv2d(16, 4, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # decoder layers #
        self.conv4 = nn.Conv2d(4, 16, 3, padding = 1)
        self.conv5 = nn.Conv2d(16, 1, 3, padding = 1)
        
    def forward(self, x):
        # encoder #
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        
        # decoder #
        x = F.interpolate(x, scale_factor = 2, mode = 'nearest')
        x = F.relu(self.conv4(x))
        
        x = F.interpolate(x, scale_factor = 2, mode = 'nearest')
        x = F.sigmoid(self.conv5(x))
        
        return x
    
model = ConvAutoencoder()
print(model)
        

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

n_epochs = 30

for epoch in range(1, n_epochs + 1):
    train_loss = 0.0
    
    for data in trainLoader:
        images, _ = data
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
        
    train_loss = train_loss/len(trainLoader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

In [ ]:
dataiter = iter(testLoader)
images, labels = dataiter.next()

output = model(images)
images = images.numpy()

output = output.view(batchSize, 1, 28, 28)
output = output.detach().numpy()

fig, axes = plt.subplots(nrows = 2, ncols = 10, sharex = True, sharey = True, figsize = (25, 4))

for images, row in zip([images, output], axes):
    for img, ax in zip(images, row):
        ax.imshow(np.squeeze(img), cmap = 'gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

***

Denoising autoencoder

As I've mentioned before, autoencoders like the ones you've built so far aren't too useful in practive. However, they can be used to denoise images quite successfully just by training the network on noisy images. We can create the noisy images ourselves by adding Gaussian noise to the training images, then clipping the values to be between 0 and 1.

* We'll use noisy images as input and the original, clean images as targets.

* Try with different types of noises

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvDenoiser(nn.Module):
    def __init__(self):
        super(ConvDenoiser, self).__init__()
        
        # encoder #
        self.conv1 = nn.Conv2d(1, 32, 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 16, 3, padding = 1)
        self.conv3 = nn.Conv2d(16, 8, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        
        # decoder #
        self.t_conv1 = nn.ConvTranspose2d(8, 8, 3, stride = 2)
        self.t_conv2 = nn.ConvTranspose2d(8, 16, 2, stride = 2)
        self.t_conv3 = nn.ConvTranspose2d(16, 32, 2, stride = 2)
        self.conv_out = nn.Conv2d(32, 1, 3, padding = 1)
        
    def forward(self, x):
        # encode #
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        
        # decode #
        x = F.relu(self.t_conv1(x))
        x = F.relu(self.t_conv2(x))
        x = F.relu(self.t_conv3(x))
        x = F.sigmoid(self.conv_out(x))
        
        return x
    
model = ConvDenoiser()
print(model)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

n_epochs = 20
noise_factor = 0.5

for epoch in range(1, n_epochs + 1):
    train_loss = 0.0
    
    for data in trainLoader:
        images, _ = data
        
        noisy_imgs = images + noise_factor * torch.randn(*images.shape)
        noisy_imgs = np.clip(noisy_imgs, 0., 1.)
        
        optimizer.zero_grad()
        outputs = model(noisy_imgs)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
        
    train_loss = train_loss/len(trainLoader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

***

Linear Autoencoder

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        #encoder
        self.fc1 = nn.Linear(28*28, encoding_dim)
        
        #decoder
        self.fc2 = nn.Linear(encoding_dim, 28*28)
        
    def forward(self, x):
        # encoding
        x = F.relu(self.fc1(x))
        
        # decoding
        x = torch.sigmoid(self.fc2(x))
        return x
    
encoding_dim = 32
model = Autoencoder(encoding_dim)
print(model)

In [ ]:
n_epochs = 20
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(1, n_epochs + 1):
    train_loss = 0.0
    
    for data in trainLoader:
        images, _ = data
        images = images.view(images.size(0), -1)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
        
    train_loss = train_loss / len(trainLoader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))